In [46]:


import os, pandas as pd
# Add asset from file system
df_data_1 = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/TSLA.csv')
df_data_1.head()



,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-05-26,45.531250,45.71875,45.312500,45.718750,35.550995,74800
1,2000-05-30,46.375000,47.40625,46.375000,47.406250,36.863197,57600
2,2000-05-31,47.562500,48.18750,47.562500,47.578125,36.996853,36000
3,2000-06-01,48.554688,48.65625,48.554688,48.656250,37.835213,7000
4,2000-06-02,50.859375,51.18750,50.859375,51.187500,39.803505,29400


In [51]:
df=df_data_1
df.shape

(4851, 7)

In [48]:
#load model
from keras.models import load_model
regressor_mae = load_model(filepath="my_model_with_mae_60_ts.h5")

In [49]:
def get_test_length(dataset, batch_size):
    
    test_length_values = []
    for x in range(len(dataset) - 200, len(dataset) - TIME_STEPS*2): 
        modulo=(x-upper_train)%batch_size
        if (modulo == 0):
            test_length_values.append(x)
            
    return (max(test_length_values))

def get_train_length(dataset, batch_size, test_percent):
    # substract test_percent to be excluded from training, reserved for testset
    length = len(dataset)
    length *= 1 - test_percent
    train_length_values = []
    for x in range(int(length) - 100,int(length)): 
        modulo=x%batch_size
        if (modulo == 0):
            train_length_values.append(x)
            
    return (max(train_length_values))

In [50]:
# defining the batch size and number of epochs
BATCH_SIZE = 64
epochs = 120
TIME_STEPS = 60

In [52]:
length = get_train_length(df, BATCH_SIZE, 0.1)
print(length)


4352


In [53]:
#Adding timesteps * 2
upper_train = length + TIME_STEPS*2
train_cols = ["Open","High","Low","Close","Volume"]
df_train_set = df.loc[:,train_cols].values
df_train_set.shape

(4851, 5)

In [54]:
test_length = get_test_length(df, BATCH_SIZE)
print(test_length)
upper_test = test_length + TIME_STEPS*2
testset_length = test_length - upper_train
print(testset_length)

4728
256


In [58]:
# construct test set
from sklearn.preprocessing import MinMaxScaler
import numpy as np
sc = MinMaxScaler(feature_range = (0, 1))

#scaling
scaled_real_bcg_values_test = sc.fit_transform(np.float64(df_train_set))

#creating input data
X_test = []
for i in range(TIME_STEPS, testset_length + TIME_STEPS):
    X_test.append(scaled_real_bcg_values_test[i-TIME_STEPS:i, 0])
X_test = np.array(X_test)


#reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],, 1))


SyntaxError: invalid syntax (<ipython-input-58-ab2dcf1ca87e>, line 17)

In [59]:
X_test.shape

(256, 60)

In [45]:
# Feature Scaling
#scale between 0 and 1. the weights are esier to find.
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(np.float64(training_set))
training_set_scaled.shape

(4851, 1)

In [60]:
#prediction
predicted_bcg_values_test_mae = regressor_mae.predict(X_test, batch_size=BATCH_SIZE)
regressor_mae.reset_states()

print(predicted_bcg_values_test_mae.shape)

#reshaping
predicted_bcg_values_test_mae = np.reshape(predicted_bcg_values_test_mae, 
                                       (predicted_bcg_values_test_mae.shape[0], 
                                        predicted_bcg_values_test_mae.shape[1]))

print(predicted_bcg_values_test_mae.shape)
#inverse transform
predicted_bcg_values_test_mae = sc.inverse_transform(predicted_bcg_values_test_mae)


#creating y_test data
y_test = []
for j in range(0, testset_length - timesteps):
    y_test = np.append(y_test, predicted_bcg_values_test_mae[j, TIME_STEPS-1])

# reshaping
y_test = np.reshape(y_test, (y_test.shape[0], 1))

print(y_test.shape)

ValueError: Error when checking : expected lstm_2_input to have 3 dimensions, but got array with shape (256, 60)

In [64]:
!ls outputs/